# Systems Immunogenetics Project

## WNV Cleaning Steps

### McWeeney Lab, Oregon Health & Science University

** Authors: Gabrielle Choonoo (choonoo@ohsu.edu) and Michael Mooney (mooneymi@ohsu.edu) **

## Introduction

This is the step-by-step process for annotating phenotypes based on clinical score and weight data.

Required Files:
* WNV Data: Processed Weight and clinical score
* This notebook (phenotypes.ipynb): [[Download here]](https://raw.githubusercontent.com/gchoonoo/WNV_cleaning_scripts/master/phenotypes.ipynb)

Required R packages:
- `gdata`
- `R.utils`

**Note: this notebook can also be downloaded as an R script (only the code blocks seen below will be included): [[Download R script here]](https://raw.githubusercontent.com/gchoonoo/WNV_cleaning_scripts/master/phenotypes.r)

** All code is available on GitHub: [https://github.com/gchoonoo/WNV_cleaning_scripts](https://github.com/gchoonoo/WNV_cleaning_scripts) **

# Read in latest data release

In [1]:
library(gdata)
library(R.utils)

read.xls(xls="./Lund_Weight_22-Mar-2016_final.xlsx", sheet=1) -> lund_weight
read.xls(xls="./Lund_Scores_22-Mar-2016_final.xlsx", sheet=1) -> lund_score
read.xls(xls="./Gale_Weight_22-Mar-2016_final.xlsx", sheet=1) -> gale_weight
read.xls(xls="./Gale_Scores_22-Mar-2016_final.xlsx", sheet=1) -> gale_score

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

Loading required package: R.oo
Loading required package: R.methodsS3
R.methodsS3 v1.7.1 (2016-02-15) successfully loaded. See ?R.methodsS3 for help.
R.oo v1.20.0 (2016-02-17) successfully loaded. See ?R.oo for help.

Attaching package: ‘R.oo’

The following objects are masked from ‘package:gdata’:

    ll, trim

The following objects are masked from ‘package:methods’:

    getClasses, getMethods

The following objects are masked from ‘package:base’:

    attach, detach, gc, load, save

R.utils v2.3.0 (2016-04-13) successfully loaded. See ?R.utils for help.

Attaching package: ‘R.utils’

The following objects are masked from ‘package:gdata’:

    env, resample

The following object is masked from ‘

# Merge Lund Weight and CS Data

In [2]:
## Fix CS column names
days = c("D0", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", "D13", "D14", "D15", "D16", "D17", "D18", "D19", "D20", "D21", "D22", "D23", "D24", "D25", "D26", "D27", "D28")
colnames(lund_score)[colnames(lund_score) %in% days] = paste(colnames(lund_score)[colnames(lund_score) %in% days], "cs", sep="_")

cs_days = paste(days,"cs",sep="_")

lund_score[,c("ID","Mating","RIX_ID","UW_Line","Virus","Date_Infected","Death_Date","Timepoint","Lab",cs_days)] -> lund_score_subset

lund_weight[,c("ID","Mating","RIX_ID","UW_Line","Virus","Date_Infected","Death_Date","Timepoint","Lab",days,paste0(days,"_Percentage"))] -> lund_weight_subset

merge_cols = intersect(names(lund_weight_subset), names(lund_score_subset))

all_lund_weight_cs = merge(lund_weight_subset, lund_score_subset, by=merge_cols, all=T)

# Merge Gale Weight and CS Data

In [3]:
## Fix CS column names
colnames(gale_score)[colnames(gale_score) %in% days] = paste(colnames(gale_score)[colnames(gale_score) %in% days], "cs", sep="_")

gale_score[,c("ID","Mating","RIX_ID","UW_Line","Virus","Date_Infected","Death_Date","Timepoint","Lab",cs_days)] -> gale_score_subset

gale_weight[,c("ID","Mating","RIX_ID","UW_Line","Virus","Date_Infected","Death_Date","Timepoint","Lab",days,paste0(days,"_Percentage"))] -> gale_weight_subset

all_gale_weight_cs = merge(gale_weight_subset, gale_score_subset, by=merge_cols, all=T)

### Merge both labs
all_weight_cs = rbind(all_lund_weight_cs, all_gale_weight_cs)

# Calculate Phenotype criteria (max clinical score, average weight loss)

## Calculate Max Clinical Scores

In [4]:
cs_days7 = c("D0_cs","D1_cs","D2_cs", "D3_cs", "D4_cs", "D5_cs", "D6_cs","D7_cs")
cs_days8_14 = c("D8_cs", "D9_cs", "D10_cs", "D11_cs","D12_cs","D13_cs","D14_cs")
cs_days15_28 = c("D15_cs", "D16_cs", "D17_cs", "D18_cs", "D19_cs", "D20_cs","D21_cs", "D22_cs", "D23_cs", "D24_cs", "D25_cs", "D26_cs", "D27_cs", "D28_cs")

all_weight_cs$max_cs = suppressWarnings(apply(all_weight_cs[,cs_days], 1, max, na.rm=T))
all_weight_cs$max_cs_d7 = suppressWarnings(apply(all_weight_cs[,cs_days7], 1, max, na.rm=T))
all_weight_cs$max_cs_d8_d14 = suppressWarnings(apply(all_weight_cs[,cs_days8_14], 1, max, na.rm=T))
all_weight_cs$max_cs_d15_d28 = suppressWarnings(apply(all_weight_cs[,cs_days15_28], 1, max, na.rm=T))

all_weight_cs$max_cs[is.infinite(all_weight_cs$max_cs)] = NA
all_weight_cs$max_cs_d7[is.infinite(all_weight_cs$max_cs_d7)] = NA
all_weight_cs$max_cs_d8_d14[is.infinite(all_weight_cs$max_cs_d8_d14)] = NA
all_weight_cs$max_cs_d15_d28[is.infinite(all_weight_cs$max_cs_d15_d28)] = NA

per_day_weight_change = function(w, start_day=1) {
  #print(w)
  max_day = max(which(!is.na(w)))
  day_span = max_day+start_day-1
  if (!is.infinite(max_day) & day_span > 1) {
    return(w[max_day]/(day_span-1))
  } else {
    return(NA)
  }
}

days7 = c("D0", "D1", "D2", "D3", "D4", "D5", "D6", "D7")
days8_14 = c("D8", "D9", "D10", "D11", "D12", "D13", "D14")
days15_28 = c("D15", "D16", "D17", "D18", "D19", "D20", "D21", "D22", "D23", "D24", "D25", "D26", "D27", "D28")

days_percent = paste0(days[-1],"_Percentage")
days7_percent = paste0(days7[-1],"_Percentage")
days8_14_percent = paste0(days8_14,"_Percentage")
days15_28_percent = paste0(days15_28,"_Percentage")

all_weight_cs$avg_w_change = suppressWarnings(apply(all_weight_cs[,days_percent], 1, per_day_weight_change))
all_weight_cs$avg_w_change_d7 = suppressWarnings(apply(all_weight_cs[,days7_percent], 1, per_day_weight_change))
all_weight_cs$avg_w_change_d8_d14 = suppressWarnings(apply(all_weight_cs[,days8_14_percent], 1, per_day_weight_change, start_day=8))
all_weight_cs$avg_w_change_d15_d28 = suppressWarnings(apply(all_weight_cs[,days15_28_percent], 1, per_day_weight_change, start_day=15))

all_weight_cs$max_w_change = suppressWarnings(apply(all_weight_cs[,days_percent], 1, min, na.rm=T))
all_weight_cs$max_w_change_d7 = suppressWarnings(apply(all_weight_cs[,days7_percent], 1, min, na.rm=T))
all_weight_cs$max_w_change_d8_d14 = suppressWarnings(apply(all_weight_cs[,days8_14_percent], 1, min, na.rm=T))
all_weight_cs$max_w_change_d15_d28 = suppressWarnings(apply(all_weight_cs[,days15_28_percent], 1, min, na.rm=T))

all_weight_cs$max_w_change[is.infinite(all_weight_cs$max_w_change)] = NA
all_weight_cs$max_w_change_d7[is.infinite(all_weight_cs$max_w_change_d7)] = NA
all_weight_cs$max_w_change_d8_d14[is.infinite(all_weight_cs$max_w_change_d8_d14)] = NA
all_weight_cs$max_w_change_d15_d28[is.infinite(all_weight_cs$max_w_change_d15_d28)] = NA


## Aggregate average weight loss by Line and Virus

In [5]:
## Before D8
all_weight_cs$d7_w_change_gt5percent_mock = NA
agg_w_change = aggregate(all_weight_cs$avg_w_change_d7, by=list(all_weight_cs$UW_Line, all_weight_cs$Virus), mean, na.rm=T)
colnames(agg_w_change) = c('line', 'virus', 'w_change')

lines = unique(agg_w_change$line)
for (l in lines) {
  mock_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='Mock', 'w_change']
  wnv_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='WNV', 'w_change']
  if (length(mock_change) > 0 & length(wnv_change) > 0) { 
    if (!is.na(mock_change) & !is.na(wnv_change)) {
      delta = wnv_change-mock_change
      #print(l)
      #print(delta)
      if (delta < -5) {
        all_weight_cs$d7_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d7_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}

## D8 - D14
all_weight_cs$d8_d14_w_change_gt5percent_mock = NA
agg_w_change = aggregate(all_weight_cs$avg_w_change_d8_d14, by=list(all_weight_cs$UW_Line, all_weight_cs$Virus), mean, na.rm=T)
colnames(agg_w_change) = c('line', 'virus', 'w_change')

lines = unique(agg_w_change$line)
for (l in lines) {
  mock_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='Mock', 'w_change']
  wnv_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='WNV', 'w_change']
  if (length(mock_change) > 0 & length(wnv_change) > 0) { 
    if (!is.na(mock_change) & !is.na(wnv_change)) {
      delta = wnv_change-mock_change
      #print(l)
      #print(delta)
      if (delta < -5) {
        all_weight_cs$d8_d14_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d8_d14_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}

## After D14
all_weight_cs$d15_d28_w_change_gt5percent_mock = NA
agg_w_change = aggregate(all_weight_cs$avg_w_change_d15_d28, by=list(all_weight_cs$UW_Line, all_weight_cs$Virus), mean, na.rm=T)
colnames(agg_w_change) = c('line', 'virus', 'w_change')

lines = unique(agg_w_change$line)
for (l in lines) {
  mock_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='Mock', 'w_change']
  wnv_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='WNV', 'w_change']
  if (length(mock_change) > 0 & length(wnv_change) > 0) { 
    if (!is.na(mock_change) & !is.na(wnv_change)) {
      delta = wnv_change-mock_change
      #print(l)
      #print(delta)
      if (delta < -5) {
        all_weight_cs$d15_d28_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d15_d28_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}


## Aggregate max weight loss by Line and Virus

In [6]:
## Before D8
all_weight_cs$d7_max_w_change_gt5percent_mock = NA
agg_w_change = aggregate(all_weight_cs$max_w_change_d7, by=list(all_weight_cs$UW_Line, all_weight_cs$Virus), mean, na.rm=T)
colnames(agg_w_change) = c('line', 'virus', 'w_change')

lines = unique(agg_w_change$line)
for (l in lines) {
  mock_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='Mock', 'w_change']
  wnv_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='WNV', 'w_change']
  if (length(mock_change) > 0 & length(wnv_change) > 0) { 
    if (!is.na(mock_change) & !is.na(wnv_change)) {
      delta = wnv_change-mock_change
      #print(l)
      #print(delta)
      if (delta < -5) {
        all_weight_cs$d7_max_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d7_max_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}

## D8 - D14
all_weight_cs$d8_d14_max_w_change_gt5percent_mock = NA
agg_w_change = aggregate(all_weight_cs$max_w_change_d8_d14, by=list(all_weight_cs$UW_Line, all_weight_cs$Virus), mean, na.rm=T)
colnames(agg_w_change) = c('line', 'virus', 'w_change')

lines = unique(agg_w_change$line)
for (l in lines) {
  mock_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='Mock', 'w_change']
  wnv_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='WNV', 'w_change']
  if (length(mock_change) > 0 & length(wnv_change) > 0) { 
    if (!is.na(mock_change) & !is.na(wnv_change)) {
      delta = wnv_change-mock_change
      #print(l)
      #print(delta)
      if (delta < -5) {
        all_weight_cs$d8_d14_max_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d8_d14_max_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}

## After D14
all_weight_cs$d15_d28_max_w_change_gt5percent_mock = NA
agg_w_change = aggregate(all_weight_cs$max_w_change_d15_d28, by=list(all_weight_cs$UW_Line, all_weight_cs$Virus), mean, na.rm=T)
colnames(agg_w_change) = c('line', 'virus', 'w_change')

lines = unique(agg_w_change$line)
for (l in lines) {
  mock_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='Mock', 'w_change']
  wnv_change = agg_w_change[agg_w_change$line==l & agg_w_change$virus=='WNV', 'w_change']
  if (length(mock_change) > 0 & length(wnv_change) > 0) { 
    if (!is.na(mock_change) & !is.na(wnv_change)) {
      delta = wnv_change-mock_change
      #print(l)
      #print(delta)
      if (delta < -5) {
        all_weight_cs$d15_d28_max_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d15_d28_max_w_change_gt5percent_mock[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}


## Aggregate max CS by Line

In [7]:
## Before D8
all_weight_cs$d7_max_cs_gt1 = NA
agg_max_cs = suppressWarnings(aggregate(all_weight_cs$max_cs_d7[all_weight_cs$Virus=='WNV'], by=list(all_weight_cs$UW_Line[all_weight_cs$Virus=='WNV']), max, na.rm=T))
colnames(agg_max_cs) = c('line', 'max_cs')

lines = unique(agg_max_cs$line)
for (l in lines) {
  max_cs = agg_max_cs[agg_max_cs$line==l, 'max_cs']
  if (length(max_cs) > 0) { 
    if (!is.na(max_cs)) {
      if (max_cs > 1) {
        all_weight_cs$d7_max_cs_gt1[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d7_max_cs_gt1[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}

## Between D8 and D14
all_weight_cs$d8_d14_max_cs_gt1 = NA
agg_max_cs = suppressWarnings(aggregate(all_weight_cs$max_cs_d8_d14[all_weight_cs$Virus=='WNV'], by=list(all_weight_cs$UW_Line[all_weight_cs$Virus=='WNV']), max, na.rm=T))
colnames(agg_max_cs) = c('line', 'max_cs')

lines = unique(agg_max_cs$line)
for (l in lines) {
  max_cs = agg_max_cs[agg_max_cs$line==l, 'max_cs']
  if (length(max_cs) > 0) { 
    if (!is.na(max_cs)) {
      if (max_cs > 1) {
        all_weight_cs$d8_d14_max_cs_gt1[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d8_d14_max_cs_gt1[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}

## After D14
all_weight_cs$d15_d28_max_cs_gt1 = NA
agg_max_cs = suppressWarnings(aggregate(all_weight_cs$max_cs_d15_d28[all_weight_cs$Virus=='WNV'], by=list(all_weight_cs$UW_Line[all_weight_cs$Virus=='WNV']), max, na.rm=T))
colnames(agg_max_cs) = c('line', 'max_cs')

lines = unique(agg_max_cs$line)
for (l in lines) {
  max_cs = agg_max_cs[agg_max_cs$line==l, 'max_cs']
  if (length(max_cs) > 0) { 
    if (!is.na(max_cs)) {
      if (max_cs > 1) {
        all_weight_cs$d15_d28_max_cs_gt1[all_weight_cs$UW_Line==l] = TRUE
      } else {
        all_weight_cs$d15_d28_max_cs_gt1[all_weight_cs$UW_Line==l] = FALSE
      }
    }
  }
}


# Annotate phenotypes based on criteria

In [8]:
all_weight_cs$phenotype = NA
all_weight_cs$phenotype[is.na(all_weight_cs$d7_max_cs_gt1) & is.na(all_weight_cs$d8_d14_max_cs_gt1) & is.na(all_weight_cs$d15_d28_max_cs_gt1) & is.na(all_weight_cs$d7_max_w_change_gt5percent_mock) & is.na(all_weight_cs$d8_d14_max_w_change_gt5percent_mock) & is.na(all_weight_cs$d15_d28_max_w_change_gt5percent_mock)] = 'unknown'
all_weight_cs$phenotype[any(!all_weight_cs$d7_max_cs_gt1 & !all_weight_cs$d8_d14_max_cs_gt1 & !all_weight_cs$d15_d28_max_cs_gt1 & !all_weight_cs$d7_max_w_change_gt5percent_mock & !all_weight_cs$d8_d14_max_w_change_gt5percent_mock & !all_weight_cs$d15_d28_max_w_change_gt5percent_mock) & !any(all_weight_cs$d7_max_cs_gt1 & all_weight_cs$d8_d14_max_cs_gt1 & all_weight_cs$d15_d28_max_cs_gt1 & all_weight_cs$d7_max_w_change_gt5percent_mock & all_weight_cs$d8_d14_max_w_change_gt5percent_mock & all_weight_cs$d15_d28_max_w_change_gt5percent_mock)] = 'no disease'
all_weight_cs$phenotype[all_weight_cs$d7_max_cs_gt1 | all_weight_cs$d7_max_w_change_gt5percent_mock] = 'early susceptibility'
all_weight_cs$phenotype[(!all_weight_cs$d7_max_cs_gt1 | is.na(all_weight_cs$d7_max_cs_gt1)) & (!all_weight_cs$d7_max_w_change_gt5percent_mock | is.na(all_weight_cs$d7_max_w_change_gt5percent_mock)) & (all_weight_cs$d8_d14_max_cs_gt1 | all_weight_cs$d8_d14_max_w_change_gt5percent_mock) & (!all_weight_cs$d15_d28_max_cs_gt1 | is.na(all_weight_cs$d15_d28_max_cs_gt1)) & (!all_weight_cs$d15_d28_max_w_change_gt5percent_mock | is.na(all_weight_cs$d15_d28_max_w_change_gt5percent_mock))] = 'normal disease'
all_weight_cs$phenotype[(all_weight_cs$d15_d28_max_cs_gt1 | all_weight_cs$d15_d28_max_w_change_gt5percent_mock) & (!all_weight_cs$d7_max_cs_gt1 | is.na(all_weight_cs$d7_max_cs_gt1)) & (!all_weight_cs$d7_max_w_change_gt5percent_mock | is.na(all_weight_cs$d7_max_w_change_gt5percent_mock))] = 'extended disease'

# Change line 4 back to no disease, score is outlier
all_weight_cs[which(all_weight_cs[,"UW_Line"] == '4'),"phenotype"] <- "no disease"

# Save phenotype list
pheno_list = list("no disease" = sort(unique(all_weight_cs$UW_Line[all_weight_cs$phenotype=='no disease'])),'early susceptibility'=sort(unique(all_weight_cs$UW_Line[all_weight_cs$phenotype=='early susceptibility'])),'normal disease'=sort(unique(all_weight_cs$UW_Line[all_weight_cs$phenotype=='normal disease'])),'extended disease'=sort(unique(all_weight_cs$UW_Line[all_weight_cs$phenotype=='extended disease'])))

print(pheno_list)

$`no disease`
 [1]   2   4   7  11  12  14  15  16  17  18  21  22  25  26  31  35  37  42  43
[20]  44  45  57  58  62  63  64  71  73  74  77  79  85  86  88  89  91  92  98
[39]  99 104 105

$`early susceptibility`
[1] 68 90

$`normal disease`
 [1]   3  19  20  23  27  28  29  40  46  49  52  53  55  61  65  66  70  72  80
[20]  93  94  95  96  97 100 101 102 103

$`extended disease`
 [1]  8  9 10 13 24 30 32 33 34 36 38 39 41 47 48 50 54 56 59 60 67 69 75 76 78
[26] 81 82 83 84 87

